In [ ]:
import os, glob, json, csv, subprocess, sys, re, operator, itertools
import datetime
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Iterable
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository and directories

In [ ]:
userhome = os.path.expanduser("~")
repository = userhome + r'/different-diff/dataset/hbase/'

In [ ]:
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Loading comparison data between bugline and blame

In [ ]:
compare_dirs = analyze_dir + 'step_3/diff-file_blame-file_comparison/'

In [ ]:
dirs = []
for d in glob.iglob(compare_dirs + '*', recursive=True):
    dirs.append(d)
dirs.sort()
dirs

In [ ]:
column_name = ['bug_id','line_number','buggy_code','does_bugline_from_originfile?','date_added_in_originfile','earliest_affected_version','version_release_date','bug_class']

# Histogram

In [ ]:
#Reading files from comparison
result = []
for file in glob.iglob(dirs[0] + '/*', recursive=True):
    res = []
    with open(file,'r') as myfile:
        csv_reader = csv.reader(myfile, delimiter=',')
        for row in csv_reader:
            res.append(row[7])
    del res[0]
    
    regex = re.search("\_(?P<commitid>[a-z0-9]{10})\-(?P<parentid>[a-z0-9]{10})\_", file)
    regex = regex.groups()
    
    if ('bug-introducing change') not in res:
        tmp = [file,row[0],regex[0],regex[1],"Does not affect the version"]
    else:
        tmp = [file,row[0],regex[0],regex[1],"Affect the version"]
    result.append(tmp)
result.sort()

In [ ]:
result

In [ ]:
with open(analyze_dir + "step_3/affect_vs_nonaffect/affectfiles_classification.csv", 'w') as myfile:
    writers = csv.writer(myfile, delimiter=",")
    cols = ['filename','bug_id','commit_id','parent_id','affect_or_nonaffect']
    writers.writerow(cols)
    for el in result:
        writers.writerow(el)

In [ ]:
#Grouping files based on the result
affect = []
not_affect = []
for ll, mm in enumerate(result):
    if mm[2] == 'Affect the version':
        affect.append(mm)
    else:
        not_affect.append(mm)
not_affect

In [ ]:
affect[0][1]

In [ ]:
#Capturing bug ids which affected the earliest version 
bug_id_affect = []
for b, b_id in enumerate(affect):
    bug_id_affect.append(b_id[1])
bug_id_affect.sort()
print (len(bug_id_affect))
bug_id_affect

In [ ]:
#Removing duplicate bug id in bug_id_affect
affected_bug_id = list(set(bug_id_affect))
affected_bug_id.sort()
num_bid = len(affected_bug_id)
print ("Number of bug id affected the earliest version: ", num_bid)
affected_bug_id

In [ ]:
#Capturing bug ids which did not affect the earliest version 
bug_id_notaffect = []
for b, b_id in enumerate(not_affect):
    bug_id_notaffect.append(b_id[1])
bug_id_notaffect.sort()
print (len(bug_id_notaffect))
bug_id_notaffect

In [ ]:
#Removing duplicate bug id in bug_id_notaffect
notaffected_bug_id = list(set(bug_id_notaffect))
notaffected_bug_id.sort()
num_nbid = len(notaffected_bug_id)
print ("Number of bug id did not affect the earliest version: ", num_nbid)
notaffected_bug_id

In [ ]:
num_affect = len(affect)
num_notaffect = len(not_affect)

print ("Number of files affect the earliest version: %i" %(num_affect))
print ("Number of files did not affect the earliest version: %i" %(num_notaffect))

# Minimal

# Visualizing the number of affected and not affected files between algorithms

In [ ]:
graph_bid = go.Bar(
    x = ['#Bug ids affect'],
    y = [num_bid],
    text = [num_bid],
    textposition = 'auto',
    name = 'Bug ids affect'
)

graph_aff = go.Bar(
    x = ['#Files affect'],
    y = [num_affect],
    text = [num_affect],
    textposition = 'auto',
    name = 'Files affect'
)

graph_noaff = go.Bar(
    x = ['#Files do not affect'],
    y = [num_notaffect],
    text = [num_notaffect],
    textposition = 'auto',
    name = 'Files do not affect'
)


dt = [graph_bid, graph_aff, graph_noaff]

layout = go.Layout(
    title = 'Number of files which have effect to the earliest version'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)